# 要約 
このJupyter Notebookでは、ルールベースの質問者を実装する方法について説明しています。具体的には、既存のハイブリッドアプローチに基づき、複数の「if」文を使用せずに簡潔な方法でルールベースの質問者を構築することを目指しています。Markdownセルでは、ルールベースのアルゴリズムの実装に関する基本的なアイデアが紹介されています。

### 問題に取り組む内容
主な目的は、質問ゲーム「20の質問」に対応したルールベースの質問者を作成することです。多くの上位ランカーはハイブリッドなアプローチを利用している中で、著者は独自の簡潔な実装方法を示しています。

### 使用されている手法とライブラリ
1. **ルールベースの質問者の実装**:
   - `Robot`クラスを定義し、質問モード、回答モード、推測モードに対応するメソッド`on`を実装しています。
   - 質問の構造はツリー形式で辞書 (`tree_dic`) で管理され、回答履歴に基づいて次の質問を決定します。
   - 回答モードでは、ランダムに「yes」または「no」を返すように実装しています。
   - 推測モードでは、常に「apple」と予測する簡単な実装を行っています。

2. **環境の設定**:
   - `kaggle_environments`ライブラリを使用して、「20の質問」ゲームに対応するシミュレーション環境を作成しています。
   - 情報に従って、エージェントを動かし、ゲームの結果を可視化するためのコードも組まれています。

このNotebookの重要な要素は、効率的な質問アルゴリズムの実装と、Kaggleのゲーム環境でその実装をテストすることに焦点を当てている点です。また、ルールベースの質問者を強化するための追加的な方法やライブラリについての情報提供が求められています。

---


# 用語概説 
以下に、Jupyter Notebookの内容から、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説を列挙します。

1. **ルールベース**:
   ルールベースとは、特定のルールや条件に従って判断や行動を行うシステムのことです。機械学習や深層学習では、データから学習するのではなく、事前に設定されたルールによって決定を下します。このコンテキストでは、質問者が事前に定義された質問の木に基づいて動作します。

2. **質問の木**:
   質問の木とは、質問が層状に構造化された情報のフレームワークで、各質問に対する「はい」または「いいえ」の応答に基づいて次の質問を選択する仕組みです。これにより、質問者はターゲットワードを絞り込んでいきます。

3. **obs**:
   `obs`は、観察（observation）の略で、エージェントが現在の状況や環境に関する情報を持つデータオブジェクトを指します。この情報には、質問履歴やターン情報などが含まれます。

4. **answers_hist**:
   `answers_hist`は、過去の回答の履歴を保持するための変数です。この履歴を元に、次の質問を決定する際に使用されます。具体的には、過去の「はい」または「いいえ」の回答を記録します。

5. **kaggle_environmentsライブラリ**:
   Kaggle Environmentsは、Kaggleのコンペティションのために設計されたシミュレーション環境を提供するためのライブラリです。このライブラリを使用することで、エージェントの動作をシミュレートし、競技や試験を行うことができます。

6. **デバッグモード**:
   デバッグモードとは、プログラムや環境を開発時に、エラーを診断しやすくするための特別な運用モードです。このモードを有効にすると、エラーが発生した場合に詳細なエラーメッセージや状態を確認でき、問題解析が容易になります。

7. **インスタンスを作成する**:
   クラスから具体的なオブジェクトを生成するプロセスを指します。クラスは設計図であり、インスタンスはその設計に基づいて実体化されたものです。この文脈では、`Robot`クラスのインスタンスを生成しています。

8. **ゲームの結果を可視化する**:
   ゲームのシミュレーション結果をグラフィカルに表示することを指します。これにより、エージェントのパフォーマンスや動きが直感的に理解できるようになります。

これらの用語は、本ノートブックの内容を理解する上で重要であり、初心者にとっては最初は馴染みが薄いかもしれません。

---


# ルールベースの質問者をどのように実装しますか？
多くのトップランカーは、ハイブリッドのルールベースおよびLLM質問者を開発しています。ルールベースのアルゴリズムをどのように実装しますか？複数の「if」を使用しますか？その他に役立つライブラリはありますか？
私は、複数の「if」文を使用せずにルールベースの質問者を実装しました。他に簡単な方法があれば、コメントで教えてください！

![tree.png](attachment:ca56c51b-c2d5-44ee-b7d5-a199907a881f.png)

In [ ]:
%%bash
# submissionというディレクトリを作成します。
# -pオプションにより、既に存在する場合はエラーを返さずにスルーします。
mkdir -p /kaggle/working/submission

In [ ]:
%%writefile -a submission/main.py

import random

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        # modeが"asking", "guessing", "answering"のいずれかであることを確認します。
        assert mode in ["asking", "guessing", "answering"], "modeは次のいずれかの値を取る必要があります: asking, answering, guessing"
        
        # 質問モードの場合
        if mode == "asking":
            # 質問の木を辞書として定義します。
            tree_dic = {
                "1": {
                    "": "それは場所ですか？",
                },
                "2": {
                    "y": "それは国ですか？",
                    "n": "それは食べ物ですか？",
                },
                "3": {
                    "yy":"それはヨーロッパにありますか？",
                    "yn":"それは都市ですか？",
                    "ny":"その食べ物は甘いですか？",
                    "nn":"それは飲み物ですか？",
                },
                "4": {
                    "yyy":"そのキーワードは「a」から始まりますか？",
                    "yyn":"そのキーワードは「a」から始まりますか？",
                    "yny":"そのキーワードは「a」から始まりますか？",
                    "ynn":"そのキーワードは「a」から始まりますか？",
                    "nyy":"そのキーワードは「a」から始まりますか？",
                    "nyn":"そのキーワードは「a」から始まりますか？",
                    "nny":"そのキーワードは「a」から始まりますか？",
                    "nnn":"そのキーワードは「a」から始まりますか？",
                },
                "5": {
                    "yyyy":"",
                    "yyyn":"そのキーワードは「b」から始まりますか？",
                    "yyny":"",
                    "yynn":"そのキーワードは「b」から始まりますか？",
                    "ynyy":"",
                    "ynyn":"そのキーワードは「b」から始まりますか？",
                    "ynny":"",
                    "ynnn":"そのキーワードは「b」から始まりますか？",
                    "nyyy":"",
                    "nyyn":"そのキーワードは「b」から始まりますか？",
                    "nyny":"",
                    "nynn":"そのキーワードは「b」から始まりますか？",
                    "nnyy":"",
                    "nnyn":"そのキーワードは「b」から始まりますか？",
                    "nnny":"",
                    "nnnn":"そのキーワードは「b」から始まりますか？",
                }
            }
            answers_hist = ""
            # obs.answersから回答履歴を作成します。
            for answer in obs.answers:
                answers_hist += answer[0]
                
            # 回答数がツリー辞書の長さより少ない場合
            if len(obs.answers) < len(tree_dic):
                output = tree_dic[str(len(obs.answers) + 1)][answers_hist]
                # 出力が空の場合は、LLMエージェントを呼ぶ指示を返します。
                if output == "":
                    output = "LLMエージェントを呼んでください"
            else:
                output = "LLMエージェントを呼んでください"
        
        # 回答モードの場合
        if mode == "answering":
            # 'yes'または'no'からランダムに選択します。
            output = random.choice(['yes', 'no'])
        
        # 推測モードの場合
        if mode == "guessing":
            # 確定的な推測を返します。
            output = 'apple'
        
        return output

# Robotクラスのインスタンスを作成します。
robot = Robot()
    
def agent(obs, cfg):
    # obs.turnTypeに応じたレスポンスを取得します。
    if obs.turnType == "ask":
        response = robot.on(mode="asking", obs=obs)
    elif obs.turnType == "guess":
        response = robot.on(mode="guessing", obs=obs)
    elif obs.turnType == "answer":
        response = robot.on(mode="answering", obs=obs)
        
    return response

In [ ]:
%%time

# kaggle_environmentsライブラリから環境を作成します。
from kaggle_environments import make

# エージェントのパスを指定します。
agent = "/kaggle/working/submission/main.py"

# "llm_20_questions"というゲーム環境を作成し、デバッグモードを有効にします。
env = make("llm_20_questions", debug=True)

# 環境を実行し、エージェントを4つ使用します。
game_output = env.run(agents=[agent, agent, agent, agent])

# ゲームの結果を可視化します。
env.render(mode="ipython", width=600, height=500)